# Applied Data Science Capstone

*determine neighborhoods on the other side of the city that are exactly the same as your current neighborhood*

In [20]:
import pandas as pd
import numpy as np

## Segmenting and Clustering Neighborhoods in Toronto

In [10]:
from urllib.request import Request, urlopen
from urllib.error import URLError

data_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

req = Request(data_url)

try:
    with urlopen(req) as response:
        page_bytes = response.read()
        page = page_bytes.decode("utf8")
except URLError as e:
    if hasattr(e, 'reason'):
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
    elif hasattr(e, 'code'):
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)    
        

In [16]:
from html.parser import HTMLParser

class TableParser(HTMLParser):
    
    def __init__(self):
        
        HTMLParser.__init__(self)
        self._header = False
        self._cell = False
        self.tables = []
        self._current_cell = []
        self._current_row = []
        self._data_separator=' '
        self._current_table = []
        
    def handle_starttag(self, tag, attrs):
        if(tag == 'td'):
            self._cell = True
        if(tag == 'th'):
            self._header = True
            
    def handle_data(self, data):
        if(self._header or self._cell):
            self._current_cell.append(data.strip())
            
    def handle_charref(self, name):
        
        if self._parse_html_entities:
            self.handle_data(self.unescape('&#{};'.format(name)))
            
    def handle_endtag(self, tag):
        """ Here we exit the tags. If the closing tag is </tr>, we know that we
        can save our currently parsed cells to the current table as a row and
        prepare for a new row. If the closing tag is </table>, we save the
        current table and prepare for a new one.
        """
        if tag == 'td':
            self._in_td = False
        elif tag == 'th':
            self._in_th = False

        if tag in ['td', 'th']:
            final_cell = self._data_separator.join(self._current_cell).strip()
            self._current_row.append(final_cell)
            self._current_cell = []
        elif tag == 'tr':
            self._current_table.append(self._current_row)
            self._current_row = []
        elif tag == 'table':
            self.tables.append(self._current_table)
            self._current_table = []


In [64]:
parser = TableParser()
parser.feed(page)

if len(parser.tables) > 0:
    table = np.array(parser.tables[0])
    df_raw = pd.DataFrame(data=table[1:,0:], columns=table[0,0:])
else:   
    print('Tables not found')

In [47]:
df_raw.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [68]:
df_raw = df_raw[df_raw.Borough != 'Not assigned']
df_raw.loc[df_raw.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df_raw.loc[df_raw.Neighbourhood == 'Not assigned', 'Borough']

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
